In [1]:
import pandas as pd
import requests
import xml.etree.ElementTree as ET
from datetime import datetime
from openpyxl import load_workbook
import openpyxl

In [2]:
# 讀取Excel檔案
keywords_df = pd.read_excel("keywords_combinations.xlsx")

In [3]:
# 設置API基本參數
user_code = '9c7a8F3961941042'  # 請替換成實際的驗證碼
base_url = 'https://gpss1.tipo.gov.tw/gpsskmc/gpss_api'

In [4]:
# 設定XML到DataFrame欄位名稱的映射
xml_to_df_mapping = {
    'publication-reference/doc-number': '公開/公告號',
    'publication-reference/date': '公開/公告日',
    'application-reference/doc-number': '申請號',
    'application-reference/date': '申請日',
    'patent-title/title': '專利名稱原文',
    'patent-title/english-title': '專利名稱英文',
    'classifications-ipc/ipc': 'IPC 分類號',
    #'classifications-cpc/cpc': 'CPC 分類號',
    'abstract/p': '摘要分段內容',
    'claims/claim/claim-text': '專利範圍段落內容'
}

In [5]:
def save_progress(index):
    with open('儲存進度.txt', 'w') as file:
        file.write(str(index))
    print(f"進度已更新，當前索引: {index}")

def load_progress():
    try:
        with open('儲存進度.txt', 'r') as file:
            return int(file.read().strip())
    except FileNotFoundError:
        return 0  # 如果沒有進度文件，從頭開始


In [6]:
def append_df_to_excel(filename, df, sheet_name='Sheet1', startrow=None, **to_excel_kwargs):
    from openpyxl import load_workbook

    # 嘗試打開現有文件
    try:
        with pd.ExcelWriter(filename, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
            # 從已有的workbook中載入所有的sheet
            writer.book = load_workbook(filename)

            # 獲取已存在的sheet的最後一行
            if startrow is None:
                try:
                    startrow = writer.sheets[sheet_name].max_row
                except KeyError:
                    # 如果指定的sheet不存在，開始行為0
                    startrow = 0
            
            # 寫入DataFrame
            df.to_excel(writer, sheet_name=sheet_name, startrow=startrow, **to_excel_kwargs)

            # 保存文件
            writer.save()
    except FileNotFoundError:
        # 如果文件不存在，則創建新文件
        df.to_excel(filename, sheet_name=sheet_name, startrow=startrow, **to_excel_kwargs)

In [7]:
# 讀取上次的進度
start_index = load_progress()

In [8]:
# 初始化存儲專利信息的列表
all_patents_info = []

In [9]:
# 遍歷Excel檔案中的每一行，從上次停止的地方開始
for index, row in keywords_df.iloc[start_index:].iterrows():
    keywords = 'TI/AB/CL=' + row['關鍵字']
    for col in ['同義詞', '上下詞']:
        if pd.notnull(row[col]):
            keywords += '&+TI/AB/CL=' + row[col]

    ipc_code = row['IPC代碼']
    api_url = f"{base_url}?userCode={user_code}&patDB=TWA,TWB,TWD&patAG=A,B&patTY=I,M,D&{keywords}&IC={ipc_code}&expFld=PN,AN,ID,AD,TI,AB,IC,CL&expFmt=xml&expQty=100"
    
    response = requests.get(api_url)
    if response.status_code == 200:
        root = ET.fromstring(response.content)
        for patentcontent in root.findall('.//patentcontent'):
            patent_dict = {df_column: patentcontent.find(f'.//{xml_path}').text if patentcontent.find(f'.//{xml_path}') is not None else '無資料'
                           for xml_path, df_column in xml_to_df_mapping.items()}
            all_patents_info.append(patent_dict)
        
        # 每處理100筆數據後儲存一次，並記錄進度
        if len(all_patents_info) >= 100:
            patents_df = pd.DataFrame(all_patents_info)
            append_df_to_excel('patents_continuous.xlsx', patents_df)
            all_patents_info = []  # 重重置列表以避免重複儲存
            save_progress(index + 1)  # 保存進度

    elif response.status_code == 429:
        print("流量用完，暫停檢索。")
        save_progress(index)  # 保存當前進度以便重新開始
        break
    else:
        print(f"請求失敗, 狀態碼: {response.status_code}")

# 處理完所有資料後，保存任何未保存的資料
if all_patents_info:
    pd.DataFrame(all_patents_info).to_excel('patents_final结果.xlsx', index=False)
    save_progress(index + 1)  # 更新進度為已完全處理

print("所有資料已處理完畢。")

C:\Users\nicon\AppData\Local\Temp\ipykernel_32876\2706685892.py:8: FutureWarning: Setting the `book` attribute is not part of the public API, usage can give unexpected or corrupted results and will be removed in a future version
  writer.book = load_workbook(filename)
C:\Users\nicon\AppData\Local\Temp\ipykernel_32876\2706685892.py:22: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


進度已更新，當前索引: 505


C:\Users\nicon\AppData\Local\Temp\ipykernel_32876\2706685892.py:8: FutureWarning: Setting the `book` attribute is not part of the public API, usage can give unexpected or corrupted results and will be removed in a future version
  writer.book = load_workbook(filename)
C:\Users\nicon\AppData\Local\Temp\ipykernel_32876\2706685892.py:22: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


進度已更新，當前索引: 510


C:\Users\nicon\AppData\Local\Temp\ipykernel_32876\2706685892.py:8: FutureWarning: Setting the `book` attribute is not part of the public API, usage can give unexpected or corrupted results and will be removed in a future version
  writer.book = load_workbook(filename)
C:\Users\nicon\AppData\Local\Temp\ipykernel_32876\2706685892.py:22: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


...

進度已更新，當前索引: 786


C:\Users\nicon\AppData\Local\Temp\ipykernel_32876\2706685892.py:8: FutureWarning: Setting the `book` attribute is not part of the public API, usage can give unexpected or corrupted results and will be removed in a future version
  writer.book = load_workbook(filename)
C:\Users\nicon\AppData\Local\Temp\ipykernel_32876\2706685892.py:22: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


進度已更新，當前索引: 796


C:\Users\nicon\AppData\Local\Temp\ipykernel_32876\2706685892.py:8: FutureWarning: Setting the `book` attribute is not part of the public API, usage can give unexpected or corrupted results and will be removed in a future version
  writer.book = load_workbook(filename)
C:\Users\nicon\AppData\Local\Temp\ipykernel_32876\2706685892.py:22: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


進度已更新，當前索引: 800
所有資料已處理完畢。


In [9]:
# 遍歷Excel檔案中的每一行，從上次停止的地方開始
for index, row in keywords_df.iloc[start_index:].iterrows():
    keywords = 'TI/AB/CL=' + row['關鍵字']
    for col in ['同義詞', '上下詞']:
        if pd.notnull(row[col]):
            keywords += '&+TI/AB/CL=' + row[col]

    ipc_code = row['IPC代碼']
    api_url = f"{base_url}?userCode={user_code}&patDB=TWA,TWB,TWD&patAG=A,B&patTY=I,M,D&{keywords}&IC={ipc_code}&expFld=PN,AN,ID,AD,TI,AB,IC,CL&expFmt=xml&expQty=100"
    
    response = requests.get(api_url)
    if response.status_code == 200:
        root = ET.fromstring(response.content)
        for patentcontent in root.findall('.//patentcontent'):
            patent_dict = {df_column: patentcontent.find(f'.//{xml_path}').text if patentcontent.find(f'.//{xml_path}') is not None else '無資料'
                           for xml_path, df_column in xml_to_df_mapping.items()}
            all_patents_info.append(patent_dict)
        
        # 每處理100筆數據後儲存一次，並記錄進度
        if len(all_patents_info) >= 100:
            patents_df = pd.DataFrame(all_patents_info)
            append_df_to_excel('patents_continuous.xlsx', patents_df)
            all_patents_info = []  # 重重置列表以避免重複儲存
            save_progress(index + 1)  # 保存進度

    elif response.status_code == 429:
        print("流量用完，暫停檢索。")
        save_progress(index)  # 保存當前進度以便重新開始
        break
    else:
        print(f"請求失敗, 狀態碼: {response.status_code}")

# 处理完所有数据后，保存任何未保存的数据
if all_patents_info:
    pd.DataFrame(all_patents_info).to_excel('patents_final结果.xlsx', index=False)
    save_progress(index + 1)  # 更新進度為已完全處理

print("所有資料已處理完畢。")

TypeError: unsupported operand type(s) for +: 'NoneType' and 'int'

In [2]:
# 讀取Excel檔案
keywords_df = pd.read_excel("keywords_combinations.xlsx")
# 設置API基本參數
user_code = ''  # 請替換成實際的驗證碼
base_url = 'https://gpss1.tipo.gov.tw/gpsskmc/gpss_api'
# 設定XML到DataFrame欄位名稱的映射
xml_to_df_mapping = {
    'publication-reference/doc-number': '公開/公告號',
    'publication-reference/date': '公開/公告日',
    'application-reference/doc-number': '申請號',
    'application-reference/date': '申請日',
    'patent-title/title': '專利名稱原文',
    'patent-title/english-title': '專利名稱英文',
    'classifications-ipc/ipc': 'IPC 分類號',
    #'classifications-cpc/cpc': 'CPC 分類號',
    'abstract/p': '摘要分段內容',
    'claims/claim/claim-text': '專利範圍段落內容'
}
def save_progress(index):
    with open('儲存進度.txt', 'w') as file:
        file.write(str(index))
    print(f"進度已更新，當前索引: {index}")

def load_progress():
    try:
        with open('儲存進度.txt', 'r') as file:
            return int(file.read().strip())
    except FileNotFoundError:
        return 0  # 如果沒有進度文件，從頭開始
def append_df_to_excel(filename, df, sheet_name='Sheet1', startrow=None, **to_excel_kwargs):
    # 打開現有的Excel文件或創建一個新的
    try:
        writer = pd.ExcelWriter(filename, engine='openpyxl', mode='a', if_sheet_exists='overlay')
        # 嘗試打開已存在的sheet
        writer.book = load_workbook(filename)
        startrow = writer.book[sheet_name].max_row
    except FileNotFoundError:
        # 文件不存在時，創建一個新的
        writer = pd.ExcelWriter(filename, engine='openpyxl', mode='w')
        startrow = 0

    # 如果startrow是None，則從開頭寫入，否則從指定的行開始寫入
    if startrow is None:
        startrow = 0

    # 寫入數據
    df.to_excel(writer, sheet_name=sheet_name, startrow=startrow, **to_excel_kwargs, index=False)

    # 保存文件
    writer.save()
# 讀取上次的進度
start_index = load_progress()
# 初始化存儲專利信息的列表
all_patents_info = []
# 遍歷Excel檔案中的每一行
for index, row in keywords_df.iloc[start_index:].iterrows():
    # 組合關鍵字查詢條件
    keywords = 'TI/AB/CL=' + row['關鍵字']
    for col in ['同義詞', '上下詞']:
        if pd.notnull(row[col]):
            keywords += '&+TI/AB/CL=' + row[col]

    ipc_code = row['IPC代碼']
    api_url = f"{base_url}?userCode={user_code}&patDB=TWA,TWB,TWD&patAG=A,B&patTY=I,M,D&{keywords}&IC={ipc_code}&expFld=PN,AN,ID,AD,TI,AB,IC,CL&expFmt=xml&expQty=100"
    
    response = requests.get(api_url)
    if response.status_code == 200:
        root = ET.fromstring(response.content)
        for patentcontent in root.findall('.//patentcontent'):
            patent_dict = {df_column: patentcontent.find(f'.//{xml_path}').text if patentcontent.find(f'.//{xml_path}') is not None else '無資料' 
                           for xml_path, df_column in xml_to_df_mapping.items()}
            all_patents_info.append(patent_dict)
        
        # 每處理100筆數據後儲存一次，並記錄進度
        if len(all_patents_info) >= 100:
            patents_df = pd.DataFrame(all_patents_info)
            patents_df.to_excel('patents_continuous.xlsx', index=False, mode='a', header=False)
            all_patents_info = []  # 重置列表以避免重複儲存
            print("數據已附加到檔案。保存進度索引:", index)

    elif response.status_code == 429:
        print("流量用完，暫停檢索。")
        break
    else:
        print(f"請求失敗, 狀態碼: {response.status_code}")
        
# 處理完所有資料後，儲存任何未保存的資料
if all_patents_info:
    pd.DataFrame(all_patents_info).to_excel('patents_final結果.xlsx', index=False)
    print("所有資料已處理完畢並保存。")